# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

Spencer Pachavis

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone 'https://github.com/spencerp77/CMSE202-s23-turnin.git'

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [35]:
#Write your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_lfw_people
import time
from sklearn.datasets import make_circles
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics

weather = pd.read_csv('aussi_weather_data.csv')
weather

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39510,2017,6,20,Darwin,19.3,33.4,0.0,6.0,11.0,22.5,...,63.0,32.0,1013.9,1010.5,0.0,1.0,24.5,32.3,0,0
39511,2017,6,21,Darwin,21.2,32.6,0.0,7.6,8.6,0.0,...,56.0,28.0,1014.6,1011.2,7.0,0.0,24.8,32.0,0,0
39512,2017,6,22,Darwin,20.7,32.8,0.0,5.6,11.0,0.0,...,46.0,23.0,1015.3,1011.8,0.0,0.0,24.8,32.1,0,0
39513,2017,6,23,Darwin,19.5,31.8,0.0,6.2,10.6,337.5,...,62.0,58.0,1014.9,1010.7,1.0,1.0,24.8,29.2,0,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [36]:
#Write your code here
weather['Location'].unique()

array(['NorfolkIsland', 'SydneyAirport', 'WaggaWagga', 'MelbourneAirport',
       'Mildura', 'Watsonia', 'Brisbane', 'Cairns', 'Townsville',
       'MountGambier', 'Nuriootpa', 'PerthAirport', 'Perth',
       'AliceSprings', 'Darwin'], dtype=object)

#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [37]:
#Write your code here
cairns = weather[weather['Location'] == 'Cairns']
cairns

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
18956,2008,12,1,Cairns,25.2,32.3,0.4,4.0,6.4,67.5,...,62.0,66.0,1011.3,1008.1,5.0,7.0,31.2,30.7,0,1
18957,2008,12,2,Cairns,24.2,32.3,8.4,6.6,5.3,67.5,...,75.0,62.0,1011.9,1008.0,7.0,7.0,29.0,31.1,1,0
18958,2008,12,3,Cairns,23.9,32.8,0.6,5.6,11.3,292.5,...,69.0,42.0,1012.2,1009.9,7.0,5.0,29.1,31.6,0,0
18959,2008,12,4,Cairns,22.2,33.6,0.0,9.6,12.3,315.0,...,60.0,31.0,1013.3,1010.9,2.0,1.0,29.6,32.5,0,0
18960,2008,12,5,Cairns,23.2,33.5,0.0,11.6,11.8,315.0,...,64.0,56.0,1013.8,1011.0,4.0,2.0,29.7,32.2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21395,2016,4,20,Cairns,22.9,30.4,0.0,7.4,6.0,315.0,...,75.0,82.0,1016.2,1013.6,6.0,7.0,26.1,26.7,0,1
21396,2016,4,25,Cairns,22.4,28.6,9.2,4.4,1.0,315.0,...,74.0,78.0,1016.7,1014.4,7.0,7.0,25.6,25.0,1,1
21397,2016,4,26,Cairns,22.0,26.9,6.0,4.4,0.9,315.0,...,89.0,87.0,1015.9,1013.5,7.0,8.0,23.6,23.1,1,1
21398,2016,4,27,Cairns,22.8,29.7,7.2,3.2,7.4,315.0,...,80.0,71.0,1014.7,1011.8,7.0,7.0,25.6,28.2,1,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [41]:
#Write your code here
X = cairns.drop(columns = ['Location', 'RainTomorrow'])
y = cairns['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .25, train_size=.75, shuffle = True)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

I think it makes more sense to create a logistic regression since this is a binary model, whethter it will rain or not tomorrow which means that a logistic model will make more sense for this type of model.  The model being a prediction model means it should be logistic.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [42]:
#Write your code here
logit_model = sm.Logit(y_train, sm.add_constant(X_train))
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.375682
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1833
Model:                          Logit   Df Residuals:                     1809
Method:                           MLE   Df Model:                           23
Date:                Fri, 21 Apr 2023   Pseudo R-squ.:                  0.3931
Time:                        15:07:58   Log-Likelihood:                -688.63
converged:                       True   LL-Null:                       -1134.6
Covariance Type:            nonrobust   LLR p-value:                1.139e-173
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -90.6659     69.156     -1.311      0.190    -226.209      44.878
Year              0.

The important factors are:

RainToday, Pressure(9am &3pm), Humidity(3pm), WindGustSpeed, Sunshine, Rainfall, Month

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [46]:
#Write your code here
y_pred = result.predict(sm.add_constant(X_test))
y_pred = np.round(y_pred)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[379  49]
 [ 61 122]]
              precision    recall  f1-score   support

           0       0.86      0.89      0.87       428
           1       0.71      0.67      0.69       183

    accuracy                           0.82       611
   macro avg       0.79      0.78      0.78       611
weighted avg       0.82      0.82      0.82       611



#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

Precision is greater since the weighted avg is the same, but precision has a slightly higher macro avg, but they are both very close.  Precision is the accuracy the model gets of saying it will rain with respect to how many times it says that it will rain in total.  Recall shows how accurate the model is at predicting it will rain with respect to the number of times it did rain.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [57]:
#Write your code for calculating the frequency of rain here.
cairns
rain = sum(cairns['RainToday'])
not_rain = len(cairns['RainToday']) - rain

print('Days it rains:', rain/len(cairns['RainToday']))
print('Days it does not rain:', not_rain / len(cairns['RainToday']))
print('If you guess it does not rain everyday, you would be correct about 70% of the time.')

Days it rains: 0.3044189852700491
Days it does not rain: 0.6955810147299509
If you guess it does not rain everyday, you would be correct about 70% of the time.


69.56%

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [ ]:
#Write your code here

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [ ]:
#Write your code here

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Write down your response here.*

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*Write down your response here.*

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*Write down your response here.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University